In [128]:
from bs4 import BeautifulSoup
import numpy
import requests
import unicodedata

In [9]:
def get_request(url):
    try:
        document = requests.get(url)
        return document.text
    except Exception as e:
        print("Exception during get request {} e".format(url))
        return None

In [18]:
html_document = get_request("https://www.giallozafferano.it/ricette-cat/page2/")
soup = BeautifulSoup(html_document, "html.parser")
cnt = soup.find_all(attrs={'class': 'gz-title'})
img_link = soup.find_all("img",attrs={'class': 'lazyload'})

print(img_link[0]['data-src'])

https://www.giallozafferano.it/images/242-24261/Muffin-con-gocce-di-cioccolato_360x300.jpg


In [129]:
def parse_recipes_pages(dictionary,page_number=1):
    html_document = get_request("https://www.giallozafferano.it/ricette-cat/page{}/".format(page_number))
    if html_document is None: 
        return None
    soup = BeautifulSoup(html_document, "html.parser")
    title_containers = soup.find_all(attrs={'class': 'gz-title'})
    
    for titles in title_containers:
        recipe_name = titles.a["title"]
        name = dict_insert_new_recipe(recipe_name,dictionary)
        dictionary[name]["Link"] = titles.a["href"]
        
def generate_name(recipe_name,dictionary):
    new_name = recipe_name
    i=0
    while(new_name in dictionary):
        new_name = recipe_name + str(i)
        i+=1
    return new_name
        
def dict_insert_new_recipe(recipe_name,dictionary):
    if recipe_name in dictionary:
        name = generate_name(recipe_name,dictionary)
        dictionary[name] = {}
        return name
    else:
        dictionary[recipe_name] = {}
        return recipe_name

In [130]:
my_dict = {}
parse_recipes_pages(my_dict,3)

In [148]:
def parse_recipe(url,dictionary):
    soup = BeautifulSoup()
    html_document = get_request(url)
    if html_document is None: 
        return None
    soup = BeautifulSoup(html_document, "html.parser")
    ingredients_cnt = soup.find_all(attrs={"class":"gz-ingredient"})
    
    dictionary["Ingredienti"] = {}
    for ingredient in ingredients_cnt:
        name = ingredient.a.text
        grm = ingredient.span.text
        grm = grm.replace("\n", "")
        grm = grm.replace("\t","")
        dictionary["Ingredienti"][name] = grm 
    steps_cnt = soup.find_all(attrs={"class":"gz-content-recipe-step"})
    steps = ""
    for step in steps_cnt:
        for match in step.findAll('span'):
            match.decompose()
        for match in step.findAll('p'):
            steps += match.text
        steps = steps.replace(" .",".")
        steps = steps.replace(" ,",",")
        steps = steps.replace(r"\'",r"'")
        
    dictionary["Steps"] = unicodedata.normalize("NFKC", steps)

In [149]:
for recipe in my_dict:
    url = my_dict[recipe]["Link"] 
    parse_recipe(url,my_dict[recipe])
my_dict

{'Sachertorte': {'Link': 'https://ricette.giallozafferano.it/Sacher-torte.html',
  'Ingredienti': {'Cioccolato fondente al 60%': '185g',
   'Tuorli': '(circa 3)60g',
   'Albumi': '(circa 3)90g',
   'Burro': 'ammorbidito65g',
   'Farina 00': '65g',
   'Zucchero a velo': '20g',
   'Zucchero': '90g',
   'Baccello di vaniglia': '1',
   'Sale fino': '1pizzico',
   'Confettura di albicocche': '150g',
   'Panna fresca liquida': '(preferibilmente con 30-35% di grassi)125g'},
  'Steps': 'Per preparare la Sachertorte come prima cosa trasferite il cioccolato in piccoli pezzi in una bastardella  e fondetelo a bagnomaria. Se preferite potete sciogliere il cioccolato in microonde azionandolo pochi secondi per volta e mescolando ad ogni interruzione in modo da non bruciare il cioccolato. Nel frattempo ponete nella ciotola di una planetaria il burro morbido, lo zucchero a velo e un pizzico di sale. Aggiungete anche i semi del baccello di vaniglia  e iniziate a lavorare il composto con la frusta  fino 